In [9]:
import json
import time
import random
import os
from bs4 import BeautifulSoup
from tqdm import tqdm

from selenium import webdriver
import numpy as np


driver = webdriver.Chrome(executable_path=r"/Users/michaelnazarian/Jupyter Projects/Scrape/chromedriver") ##path to your operadriver executable
time.sleep(20)

def get_match_ids(url, n=1):
    """Returns the match_ids for the number of months you run the for loop.

          Params:
                n: number of iterations/months (for current month and previous, pass n = 1. For current and last two, n=2 etc)
                url: url should point to the fixtures page of any league (eg: https://www.whoscored.com/Regions/252/Tournaments/2/Seasons/7811/Stages/17590/Fixtures/England-Premier-League-2019-2020)

          WARNING:/NOTE Returns matches not played out as well but in the same month.

          TODO: Nothing added to make sure the 'accept and continue' page works. Maybe add that or ensure that works.

#     """

    all_match_ids = []
    driver.get(url)
    time.sleep(random.randrange(4, 10))

    for i in range(n):
        time.sleep(random.randrange(3,12))
        els = driver.find_elements_by_xpath("//*[contains(@class, 'result-1 rc')]")
        if len(els) != 0:
            match_links = [el.get_attribute('href') for el in els]
            match_ids = [s.split("Matches/")[-1].split("/Show")[0].split("/Live", 1)[0] for s in match_links]
            all_match_ids.extend(match_ids)

        previous_el = driver.find_element_by_xpath("/html/body/div[4]/div[3]/div[1]/div[5]/div/div/a[1]/span").click()  # clicks the previous month button ##
                                                  ##/html/body/div[5]/div[2]/div[2]/div[6]/dl/dd/div/a[1]/span        (sometimes it's this one)
                                                  ##/html/body/div[5]/div[3]/div[1]/div[5]/div/div
    
        time.sleep(random.randrange(4, 6))
        
    return all_match_ids


url = "https://1xbet.whoscored.com/Regions/252/Tournaments/2/England-Premier-League" ##no need to mess with this if you're putting in match_ids manually

match_ids = get_match_ids(url=url, n=3) ##run to get match_ids
np.save("EPL_2021_22_match_ids.npy", match_ids)
              
#match_ids = [347977]

directory = "/Users/michaelnazarian/Downloads/TestFiles"
urls = [f"https://1xbet.whoscored.com/Matches/{match_id}/Live" for match_id in match_ids]


time.sleep(15)
for url in tqdm(list(set(urls))):
    match_id = url.split("Matches/")[-1].split("/Live")[0]
    time.sleep(2)
    driver.get(url)

    time.sleep(random.randrange(5, 9)) ##give the page a chance to load. This especially takes time if you have a VPN (even more so if set to some other continent). But if you don't use VPN, run the chance to get blocked. IIWII
    source = driver.page_source
    if "matchCentreData: null" not in source:
        
        sub_str = source.split("matchCentreEventTypeJson")[0].split("matchCentreData: ")[-1].strip()
        final_str = sub_str.strip().rpartition(",")[0]
        obj = json.loads(final_str) ##final event json dict

        ##save everything
        home = obj["home"]["name"]
        away = obj["away"]["name"]
        file_name = os.path.join(directory, match_id + "_" + home + "_" + away + ".json")

        with open(file_name, "w") as f:
            json.dump(obj, f)
        print("Saved")
##



  0%|          | 0/19 [00:00<?, ?it/s]

  5%|â         | 1/19 [00:12<03:45, 12.54s/it]

Saved




 11%|â         | 2/19 [00:22<03:21, 11.84s/it]

Saved




 16%|ââ        | 3/19 [00:32<03:01, 11.32s/it]

...

Saved




 95%|ââââââââââ| 18/19 [03:19<00:10, 10.80s/it]

Saved




100%|ââââââââââ| 19/19 [03:32<00:00, 11.18s/it]

Saved


In [29]:
with open(r"/Users/michaelnazarian/Downloads/TestFiles/1516644_Atletico_Chelsea.json") as f:
    data = json.load(f)

In [31]:
data.keys()

dict_keys(['playerIdNameDictionary', 'periodMinuteLimits', 'timeStamp', 'attendance', 'venueName', 'referee', 'weatherCode', 'elapsed', 'startTime', 'startDate', 'score', 'htScore', 'ftScore', 'etScore', 'pkScore', 'statusCode', 'periodCode', 'home', 'away', 'maxMinute', 'minuteExpanded', 'maxPeriod', 'expandedMinutes', 'expandedMaxMinute', 'periodEndMinutes', 'commonEvents', 'events', 'timeoutInSeconds'])

In [38]:
from pandas.io.json import json_normalize
df = json_normalize(data["events"], sep="_")
df

,id,eventId,minute,second,teamId,x,y,expandedMinute,qualifiers,satisfiedEventsTypes,...,cardType_value,cardType_displayName,relatedEventId,relatedPlayerId,goalMouthZ,goalMouthY,isShot,blockedX,blockedY,isGoal
0,2.268212e+09,2,0,0,63,0.0,0.0,0,[],[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.268212e+09,2,0,0,15,0.0,0.0,0,[],[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2.268212e+09,3,0,1,15,49.8,50.0,0,"[{'type': {'value': 213, 'displayName': 'Angle...","[90, 116, 29, 34, 36, 215, 217]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2.268213e+09,13,0,2,15,38.2,62.1,0,"[{'type': {'value': 213, 'displayName': 'Angle...","[90, 116, 29, 34, 37, 215, 217]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2.268212e+09,4,0,3,15,37.4,52.4,0,"[{'type': {'value': 212, 'displayName': 'Lengt...","[90, 116, 29, 35, 36, 215, 217]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1721,2.268281e+09,1088,96,15,15,0.0,0.0,98,[],[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1722,2.268281e+09,789,0,0,63,0.0,0.0,8,[],[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1723,2.268281e+09,1089,0,0,15,0.0,0.0,8,[],[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1724,2.268156e+09,1,0,0,15,0.0,0.0,0,"[{'type': {'value': 130, 'displayName': 'TeamF...",[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
